In [ ]:
import pandas as pd
import requests
import time

df = pd.read_csv(r"/Users/ramy/Documents/GitHub/LogicalDistillationOfGNNs/EtherScamDB/EtherScamDB-master/_data/scams.csv")
df["scam_address"] = df["scam_address"].str.replace(r"[\[\]']", "", regex=True).str.split(", ")

df = df.explode("scam_address").reset_index(drop=True)
addresses = df["scam_address"].tolist()

ETHERSCAN_API_KEY = "3GS49BKCP8XK873Y1B27MR1WSBUIVSZSRD"

def get_transactions(address):
    url = f"https://api.etherscan.io/api?module=account&action=txlist&address={address}&startblock=0&endblock=99999999&sort=desc&apikey={ETHERSCAN_API_KEY}"
    response = requests.get(url)
    data = response.json()
    
    if data["status"] == "1":
        return data["result"]
    else:
        print(f"Error fetching {address}: {data['message']}")
        return []

all_transactions = []
for address in addresses:
    transactions = get_transactions(address.strip())
    for tx in transactions:
        tx["scam_address"] = address
        all_transactions.append(tx)
    time.sleep(0.5)

tx_df = pd.DataFrame(all_transactions)
tx_df.to_csv("scam_transactions.csv", index=False)

print("Transaction data saved successfully!")

In [ ]:
scams_df = pd.read_csv(r"/Users/ramy/Documents/GitHub/LogicalDistillationOfGNNs/EtherScamDB/EtherScamDB-master/_data/scams.csv")
scams_df["scam_address"] = scams_df["scam_address"].str.replace(r"[\[\]']", "", regex=True).str.split(", ")
exploded_scams_df = scams_df.explode("scam_address")

transactions_df = pd.read_csv("/Users/ramy/Documents/GitHub/LogicalDistillationOfGNNs/scam_transactions.csv")
merged_df = pd.merge(exploded_scams_df, transactions_df, on="scam_address", how="right")